In [1]:
# Import necessary modules for scraping
from bs4 import BeautifulSoup as bs
import pandas as pd
# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time


# Scraping Notes
* HTML so far (through 3 pages) for statista.com seems to be identical in terms of tags and elements, allowing me to reuse code to access and interact with the page

# NFL Data Scraping
* Dynamic (Franchise Value 2023) https://www.statista.com/statistics/193534/franchise-value-of-national-football-league-teams-in-2010/
    
    1. click on "adjust statistic presentation button (resembles normal "settings" button), button element:
    ```
    <button aria-label="Settings" aria-haspopup="true" data-gtm="statisticGraph__icon--settings" type="button" class="toolbar__item panelCard padding-all-10 flex-s--grow" tabindex="0"><svg aria-hidden="true" height="12" width="18" class="toolbar__icon uiIcon--default"><use href="#iconAdjustmentFill"></use></svg></button>
    ```
    2. select grid option, element:
    ``` 
    <button aria-label="Display table" class="button button--active"><svg aria-hidden="true" height="16" width="16"><use href="#iconChartTableFill"></use></svg></button>
    ```
    3. Select 50 under Records drop down
    ``` 
    <select id="dataTablePageSize1717123935205"><option> <option>50</option> 
    ```
    4. data is now present in "Inspect"
    5. data is in this format in HTML 
    ```
    <span>New York Giants</span> , <span>6.8</span>```
     ```
     <tr> </tr>
     ``` 
    in table row format
    

In [2]:
# Set up WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [2]:
# 1. Dynamic (Franchise Value 2023)

# Open the website
url = 'https://www.statista.com/statistics/193534/franchise-value-of-national-football-league-teams-in-2010/'
driver.get(url)

In [3]:
# Click "Only necessary" for cookie options
# <button id="onetrust-reject-all-handler">Only necessary</button>
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[text()="Only necessary"]'))
)
button.click()


<button aria-label="Settings" aria-haspopup="true" data-gtm="statisticGraph__icon--settings" type="button" class="toolbar__item panelCard padding-all-10 flex-s--grow toolbar__item--active" tabindex="0"><svg aria-hidden="true" height="12" width="18" class="toolbar__icon uiIcon--default"><use href="#iconAdjustmentFill"></use></svg></button>



In [4]:
# Select "adjust statistic presentation" button (resembles normal "settings" button)
# <svg aria-hidden="true" height="12" width="18" class="toolbar__icon uiIcon--default"><use href="#iconAdjustmentFill"></use></svg>
adjust_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Settings"]'))
)
adjust_button.click()

In [5]:
# Select data table option in Chart Type window
table_button = driver.find_element(By.CLASS_NAME, 'button--active')

table_button.click()

In [6]:
# Element containing Login hyperlink
# <span data-gtm="conversionTeaser__anonymUserLoginLink--statisticFree" data-navigation-login-click="" class="text-underline cursor-pointer text-color--primary-contrast text-semibold">Login</span>

In [7]:
# Select Login link button 
login = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//span[@data-gtm="conversionTeaser__anonymUserLoginLink--statisticFree"]'))
)
login.click()

In [8]:
# Enter my login credentials from the config.py file (.gitignore'd)
import config

username_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'loginStat_username'))  # code working
    )
password_field = WebDriverWait(driver, 10).until(
         EC.presence_of_element_located((By.ID, 'loginStat_password')) # code working
    )
login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'loginStat_submitLogin'))     # code working
    )

# Enter credentials 
username_field.send_keys(config.USERNAME)
password_field.send_keys(config.PASSWORD)

# username_field.click()
# password_field.click()
login_button.click()

<button id="loginStat_submitLogin" type="submit" name="loginStat[submitLogin]" data-gtm="ll_login" class="button button--primary button--medium"><!----> 
                    Log in
                </button>

## User Interaction Status
- Realize after first chart selection click that an account is needed to view certain chart types
- login credential addition were successful, although button clicks are now needed again to access chart table data
- Going forward for other statista.com sites the most efficient interaction process will be:
    1. go to main statista home page
    2. login
    3. then go directly to data page through driver --> url code
- For now will just re-access data through previous button clicks ran again

In [10]:
# Select "adjust statistic presentation" button AGAIN (resembles normal "settings" button)
adjust_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Settings"]'))
)
adjust_button.click()

In [11]:
# Select data table option in Chart Type window
table_button = driver.find_element(By.XPATH, '//button[@aria-label="Display table"]')
table_button.click()

<select id="dataTablePageSize1717556325057"><option>
                    13
                </option><option>
                    25
                </option><option>
                    50
                </option></select>

In [16]:
# Select 50 record count optionin 'Records' dropdown
records_select = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'Records:')]/select"))
)
# Create select object
selection = Select(records_select)

# Select 50 records option by text
selection.select_by_visible_text('50')


<table id="data-table-1717637951971" class="dataTable"><thead><tr><th aria-controls="data-table-1717637951971" scope="col" tabindex="0" class="undefined  sorting sorting_asc"><span>Characteristic</span></th><th aria-controls="data-table-1717637951971" scope="col" tabindex="0" class="undefined  sorting"><span>Franchise value in billion U.S. dollars</span></th></tr></thead> <tbody aria-live="polite" aria-relevant="all"><tr><td class=""><!----> <span>Seattle Seahawks</span></td><td class="nowrap"><!----> <span>5</span></td></tr><tr><td class=""><!----> <span>Atlanta Falcons</span></td><td class="nowrap"><!----> <span>4.7</span></td></tr><tr><td class=""><!----> <span>Minnesota Vikings</span></td><td class="nowrap"><!----> <span>4.65</span></td></tr><tr><td class=""><!----> <span>Baltimore Ravens</span></td><td class="nowrap"><!----> <span>4.63</span></td></tr><tr><td class=""><!----> <span>Pittsburgh Steelers</span></td><td class="nowrap"><!----> <span>4.63</span></td></tr><tr><td class=""><!----> <span>Cleveland Browns</span></td><td class="nowrap"><!----> <span>4.62</span></td></tr><tr><td class=""><!----> <span>Green Bay Packers</span></td><td class="nowrap"><!----> <span>4.6</span></td></tr><tr><td class=""><!----> <span>Tennessee Titans</span></td><td class="nowrap"><!----> <span>4.4</span></td></tr><tr><td class=""><!----> <span>Indianapolis Colts</span></td><td class="nowrap"><!----> <span>4.35</span></td></tr><tr><td class=""><!----> <span>Kansas City Chiefs</span></td><td class="nowrap"><!----> <span>4.3</span></td></tr><tr><td class=""><!----> <span>Tampa Bay Buccaneers</span></td><td class="nowrap"><!----> <span>4.2</span></td></tr><tr><td class=""><!----> <span>Los Angeles Chargers</span></td><td class="nowrap"><!----> <span>4.15</span></td></tr><tr><td class=""><!----> <span>Carolina Panthers</span></td><td class="nowrap"><!----> <span>4.1</span></td></tr></tbody> <!----></table>

In [19]:
# Locate table and store in variable
table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "dataTable"))  
    )

# Extract data from table 
rows = table.find_elements(By.TAG_NAME, "tr")
table_data = []

# Iterate over table extracting table data from <td> tags
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    table_data.append(row_data)

# Print the scraped table data list
for row in table_data:
        print(row)

# NOTE team valuations are in $USD billions

[]
['Dallas Cowboys', '9']
['New England Patriots', '7']
['Los Angeles Rams', '6.9']
['New York Giants', '6.8']
['Chicago Bears', '6.3']
['Las Vegas Raiders', '6.2']
['New York Jets', '6.1']
['Washington Commanders', '6.05']
['San Francisco 49ers', '6']
['Philadelphia Eagles', '5.8']
['Miami Dolphins', '5.7']
['Houston Texans', '5.5']
['Denver Broncos', '5.1']
['Seattle Seahawks', '5']
['Atlanta Falcons', '4.7']
['Minnesota Vikings', '4.65']
['Baltimore Ravens', '4.63']
['Pittsburgh Steelers', '4.63']
['Cleveland Browns', '4.62']
['Green Bay Packers', '4.6']
['Tennessee Titans', '4.4']
['Indianapolis Colts', '4.35']
['Kansas City Chiefs', '4.3']
['Tampa Bay Buccaneers', '4.2']
['Los Angeles Chargers', '4.15']
['Carolina Panthers', '4.1']
['New Orleans Saints', '4.08']
['Jacksonville Jaguars', '4']
['Arizona Cardinals', '3.8']
['Buffalo Bills', '3.7']
['Detroit Lions', '3.6']
['Cincinnati Bengals', '3.5']


## NFL League & Team Sponsor Revenue 2010-2022
* https://www.statista.com/statistics/456355/nfl-league-team-sponsorship-revenue-worldwide/
* Will directly access without closing previous browser
    * Login credentials will not be needed again this way

In [20]:
# Open the website
league_team_rev_url = 'https://www.statista.com/statistics/456355/nfl-league-team-sponsorship-revenue-worldwide/'
driver.get(league_team_rev_url)

In [21]:
# Same code as previous site, HTML elements seem to be similar page to page
settings_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Settings"]'))
)
settings_button.click()

In [22]:
# Select data table option in Chart Type window
table_chart = driver.find_element(By.XPATH, '//button[@aria-label="Display table"]')
table_chart.click()

In [24]:
# Select 50 record count option in 'Records' dropdown
records_select = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'Records:')]/select"))
)
# Create select object
selection = Select(records_select)

# Select 50 records option by text
selection.select_by_visible_text('50')

In [25]:
# Locate table and store in variable
revenue_table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "dataTable"))  
    )

# Extract data from table 
rev_rows = revenue_table.find_elements(By.TAG_NAME, "tr")
rev_data = []

# Iterate over table extracting table data from <td> tags
for row in rev_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    rev_data.append(row_data)

# Print the scraped revenue data list
for row in rev_data:
        print(row)

# NOTE revenue numbers are in $USD millions

[]
['2022', '1,880']
['2021', '1,800']
['2020', '1,620']
['2019', '1,470']
['2018', '1,390']
['2017', '1,320']
['2016', '1,250']
['2015', '1,200']
['2014', '1,150']
['2013', '1,070']
['2012', '1,010']
['2011', '946']
['2010', '870']


In [26]:
# Close Chrome driver
driver.quit()

# NBA Data Scraping 

### More Efficient Data Access Method 
1. First access Statista.com main page
2. Enter login info
3. Then directly access desired data page
    * Eliminating need to re-code user interactions to access chart data

## Team Franchise Value 2001-2023
* https://www.statista.com/statistics/193442/average-franchise-value-in-the-nba-since-2000/

In [15]:
# Access main page of stats site
main_site = 'https://www.statista.com'
driver.get(main_site)

In [16]:
# Click "Only necessary" for cookie options

cookies = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[text()="Only necessary"]'))
)
cookies.click()

<form action="/login/"><input type="submit" value="Login" itemprop="url" data-navigation-login="" data-gtm="navMain__navigationItem--login" class="navMain__text navMain__text--button navMain__text--buttonGhost"></form>

In [17]:
# Click homepage login
login_select = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "navMain__text--buttonGhost"))
)
login_select.click()

In [18]:
# Import login credentials (gitignore'd) stored in config.py
import config

# Copied login code from NFL scraping

username_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'loginStat_username'))  # code working
    )
password_field = WebDriverWait(driver, 10).until(
         EC.presence_of_element_located((By.ID, 'loginStat_password')) # code working
    )
login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'loginStat_submitLogin'))     # code working
    )

# Enter credentials 
username_field.send_keys(config.USERNAME)
password_field.send_keys(config.PASSWORD)

# username_field.click()
# password_field.click()
login_button.click()


In [8]:
# Go directly to NBA franchise value data url
nba_franchise_value_site = 'https://www.statista.com/statistics/193442/average-franchise-value-in-the-nba-since-2000/'
driver.get(nba_franchise_value_site)

In [9]:
# Same code as previous site, HTML elements seem to be similar page to page
settings_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Settings"]'))
)
settings_button.click()

In [10]:
# Select data table option in Chart Type window
table_chart = driver.find_element(By.XPATH, '//button[@aria-label="Display table"]')
table_chart.click()

In [11]:
# Select 50 record count option in 'Records' dropdown
records_select = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'Records:')]/select"))
)
# Create select object
selection = Select(records_select)

# Select 50 records option by text
selection.select_by_visible_text('50')

In [12]:
# Locate table and store in variable
franchise_table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "dataTable"))  
    )

# Extract data from table 
franchise_rows = franchise_table.find_elements(By.TAG_NAME, "tr")
franchise_data = []

# Iterate over table extracting table data from <td> tags
for row in franchise_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    franchise_data.append(row_data)

# Print the scraped franchise value data list
for row in franchise_data:
        print(row)

# NOTE franchise value numbers are in $USD millions

[]
['2023', '3,850']
['2022', '2,863.5']
['2021', '2,480']
['2020', '2,123']
['2019', '1,868']
['2018', '1,651.5']
['2017', '1,355']
['2016', '1,245']
['2015', '1,106']
['2014', '634']
['2013', '509']
['2012', '393']
['2011', '369']
['2010', '367']
['2009', '379']
['2008', '372']
['2007', '353']
['2006', '326']
['2005', '302']
['2004', '265']
['2003', '248']
['2002', '223']
['2001', '207']


## NBA Sponsor Revenue 2010-2023
* Website: https://www.statista.com/statistics/380270/nba-sponsorship-revenue/
1. Make sure driver cell is ran to initialize chrome driver
2. Run statista main site cell then login information
3. Below site access is ready

In [7]:
# Access statista set for NBA sponsorship revenue
nba_rev_site = 'https://www.statista.com/statistics/380270/nba-sponsorship-revenue/'
driver.get(nba_rev_site)

In [8]:
# Same code as previous sites, HTML elements seem to be similar page to page for statista.com
settings_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Settings"]'))
)
settings_button.click()

In [9]:
# Select data table option in Chart Type window
table_chart = driver.find_element(By.XPATH, '//button[@aria-label="Display table"]')
table_chart.click()

In [10]:
# Select 50 record count option in 'Records' dropdown
records_select = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'Records:')]/select"))
)
# Create select object
selection = Select(records_select)

# Select 50 records option by text
selection.select_by_visible_text('50')

In [11]:
# Locate table and store in variable
nba_rev_table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "dataTable"))  
    )

# Extract data from table 
nba_rev_rows = nba_rev_table.find_elements(By.TAG_NAME, "tr")
nba_rev_data = []

# Iterate over table extracting table data from <td> tags
for row in nba_rev_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    nba_rev_data.append(row_data)

# Print the scraped nba_rev value data list
for row in nba_rev_data:
        print(row)

# NOTE NBA Sponsorship Revenue numbers are in $USD millions
# List elements in [0] season/year, [1] $ revenue amount

[]
['22/23', '1,660']
['21/22', '1,600']
['20/21', '1,460']
['19/20', '1,390']
['18/19', '1,298']
['17/18', '1,120']
['16/17', '861']
['15/16', '799']
['14/15', '739']
['13/14', '679']
['12/13', '642']
['11/12', '610']
['10/11', '572']
['09/10', '536']


In [12]:
# Close Chrome driver
# driver.quit()

# MLB Scraping 
1. Run driver intializing cell first if not already open
2. cells to open statista.com first and then login information

## MLB Franchise value as of 2024

In [19]:
#
mlb_franchise_site = 'https://www.statista.com/statistics/193637/franchise-value-of-major-league-baseball-teams-in-2010/'
driver.get(mlb_franchise_site)

In [20]:
# Same code as previous siteS, HTML elements seem to be similar page to page
settings_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Settings"]'))
)
settings_button.click()

In [21]:
# Select data table option in Chart Type window
table_chart = driver.find_element(By.XPATH, '//button[@aria-label="Display table"]')
table_chart.click()

In [22]:
# Select 50 record count option in 'Records' dropdown
records_select = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'Records:')]/select"))
)
# Create select object
selection = Select(records_select)

# Select 50 records option by text
selection.select_by_visible_text('50')

In [23]:
# Locate table and store in variable
mlb_franchise_table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "dataTable"))  
    )

# Extract data from table 
mlb_franchise_rows = mlb_franchise_table.find_elements(By.TAG_NAME, "tr")
mlb_franchise_data = []

# Iterate over table extracting table data from <td> tags
for row in mlb_franchise_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    mlb_franchise_data.append(row_data)

# Print the scraped mlb_franchise value data list
for row in mlb_franchise_data:
        print(row)

[]
['New York Yankees', '7,550']
['Los Angeles Dodgers', '5,450']
['Boston Red Sox', '4,500']
['Chicago Cubs', '4,225']
['San Francisco Giants', '3,800']
['New York Mets', '3,000']
['Philadelphia Phillies', '2,925']
['Atlanta Braves', '2,800']
['Los Angeles Angels', '2,700']
['St. Louis Cardinals', '2,550']
['Houston Astros', '2,425']
['Texas Rangers', '2,400']
['Seattle Mariners', '2,200']
['Toronto Blue Jays', '2,100']
['Chicago White Sox', '2,050']
['Washington Nationals', '2,000']
['San Diego Padres', '1,780']
['Baltimore Orioles', '1,725']
['Milwaukee Brewers', '1,605']
['Colorado Rockies', '1,475']
['Minnesota Twins', '1,460']
['Detroit Tigers', '1,450']
['Arizona Diamondbacks', '1,425']
['Cleveland Guardians', '1,350']
['Pittsburgh Pirates', '1,320']
['Cincinnati Reds', '1,255']
['Tampa Bay Rays', '1,250']
['Kansas City Royals', '1,225']
['Oakland Athletics', '1,200']
['Miami Marlins', '1,000']


In [24]:
# close the chrome driver
# driver.quit()

# NHL Scraping

## Franchise Value as of 2023, by Team
- https://www.statista.com/statistics/193732/franchise-value-of-national-hockey-league-teams-in-2010/

In [3]:
# Access main page of stats site
main_site = 'https://www.statista.com'
driver.get(main_site)

In [4]:
# Click "Only necessary" for cookie options

cookies = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[text()="Only necessary"]'))
)
cookies.click()

In [5]:
# Click homepage login
login_select = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "navMain__text--buttonGhost"))
)
login_select.click()

In [6]:
# Import login credentials (gitignore'd) stored in config.py
import config

# Copied login code from NFL scraping

username_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'loginStat_username'))  # code working
    )
password_field = WebDriverWait(driver, 10).until(
         EC.presence_of_element_located((By.ID, 'loginStat_password')) # code working
    )
login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'loginStat_submitLogin'))     # code working
    )

# Enter credentials 
username_field.send_keys(config.USERNAME)
password_field.send_keys(config.PASSWORD)

# username_field.click()
# password_field.click()
login_button.click()

In [8]:
# access NHL franchise value site once login successful
nhl_franchise_site = 'https://www.statista.com/statistics/193732/franchise-value-of-national-hockey-league-teams-in-2010/'
driver.get(nhl_franchise_site)

In [10]:
# Same code as previous siteS, HTML elements seem to be similar page to page
settings_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Settings"]'))
)
settings_button.click()

In [11]:
# Select data table option in Chart Type window
table_chart = driver.find_element(By.XPATH, '//button[@aria-label="Display table"]')
table_chart.click()

In [12]:
# Select 50 record count option in 'Records' dropdown
records_select = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'Records:')]/select"))
)
# Create select object
selection = Select(records_select)

# Select 50 records option by text
selection.select_by_visible_text('50')

In [13]:
# Locate table and store in variable
nhl_franchise_table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "dataTable"))  
    )

# Extract data from table 
nhl_franchise_rows = nhl_franchise_table.find_elements(By.TAG_NAME, "tr")
nhl_franchise_data = []

# Iterate over table extracting table data from <td> tags
for row in nhl_franchise_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    nhl_franchise_data.append(row_data)

# Print the scraped nhl_franchise value data list
for row in nhl_franchise_data:
        print(row)

[]
['Toronto Maple Leafs', '2,800']
['New York Rangers', '2,650']
['Montreal Canadiens', '2,300']
['Los Angeles Kings', '2,000']
['Boston Bruins', '1,900']
['Chicago Blackhawks', '1,875']
['Edmonton Oilers', '1,850']
['Philadelphia Flyers', '1,650']
['Washington Capitals', '1,600']
['New York Islanders', '1,550']
['New Jersey Devils', '1,450']
['Vancouver Canucks', '1,325']
['Tampa Bay Lightning', '1,250']
['Seattle Kraken', '1,225']
['Detroit Red Wings', '1,200']
['Pittsburgh Penguins', '1,175']
['Colorado Avalanche', '1,150']
['Vegas Golden Knights', '1,125']
['Calgary Flames', '1,100']
['Dallas Stars', '1,080']
['Minnesota Wild', '1,050']
['St. Louis Blues', '990']
['Nashville Predators', '975']
['Ottawa Senators', '950']
['Anaheim Ducks', '925']
['San Jose Sharks', '900']
['Carolina Hurricanes', '825']
['Winnipeg Jets', '780']
['Florida Panthers', '775']
['Columbus Blue Jackets', '765']
['Buffalo Sabres', '750']
['Arizona Coyotes', '500']
